In [1]:
import gurobipy as gp
from gurobipy import GRB
import random
import math
from itertools import combinations
import pandas as pd
# create the parameter
random.seed(1)
number_of_facility = 55 
number_of_product = 45
time_period = 20 

r_ij = {}
a_ij = {}
for i in range(0, number_of_facility ):
    for j in range(0, number_of_product):
        r_ij[i,j]=  random.randrange(12, 15) 
        a_ij[i,j] = random.randrange(2, 3)  

d_jt = {}   
c_it = {}
capa = 600
for t in range(0, time_period  ):
    for j in range(0, number_of_product):
        d_jt[j,t] = random.randrange(350, 670)  
    for i in range(0, number_of_facility  ):
        c_it[i,t] = random.randrange(500, 510)  
        
s_jk = {}
q_jk = {}
for j in range(0, number_of_product):
    for k in range(0, number_of_product):
        if j == k :
            q_jk[j,k] = 9999
            s_jk[j,k] = random.randrange(7, 15)
        else:
            s_jk[j,k] = random.randrange(7, 15)
            q_jk[j,k] = random.randrange(20, 32) 

print('Already set up the parameter')

Already set up the parameter


In [12]:
import random
import copy
ad_jt = copy.deepcopy(d_jt)
ac_it = copy.deepcopy(c_it)
w_ijt_ini={}
w_it_seq = {}

for i in range(0,number_of_facility):
    for t in range(0,time_period):
        w_it_seq[i,t] = []

for t in range(0, time_period):
    for i in range(0,number_of_facility):
        for j in range(0,number_of_product):
            if ac_it[i,t] > 0 : # remain capacity > 0
                if w_it_seq[i,t]:
                    if ad_jt[j,t]/a_ij[i,j] >= ac_it[i,t]:
                        w_ijt_ini[i,j,t] = ac_it[i,t]
                        ac_it[i,t] = ac_it[i,t]-w_ijt_ini[i,j,t]
                        w_it_seq[i,t].append(j)
                        ad_jt[j,t] = ad_jt[j,t] -(w_ijt_ini[i,j,t]*a_ij[i,j])
                    else:
                        w_ijt_ini[i,j,t]=ad_jt[j,t]/a_ij[i,j]
                        ac_it[i,t] = ac_it[i,t]-w_ijt_ini[i,j,t]
                        w_it_seq[i,t].append(j)
                        ad_jt[j,t] = 0 
                else:
                    change_over_cost = 0
                    for j in range(len(w_it_seq[i,t])):
                        if j+1 < len(w_it_seq[i,t]):
                            change_over_cost = change_over_cost + q_jk[j,j+1]
                    if (ad_jt[j,t]/a_ij[i,j] ) +change_over_cost  >= ac_it[i,t]:
                        w_ijt_ini[i,j,t] = c_it[i,t]-change_over_cost
                        w_it_seq[i,t].append(j)
                        ad_jt[j,t] = ad_jt[j,t] -(w_ijt_ini[i,j,t]*a_ij[i,j])
                        ac_it[i,t] = ac_it[i,t]-w_ijt_ini[i,j,t]
                    else:
                        w_ijt_ini[i,j,t]=ad_jt[j,t]/a_ij[i,j]
                        ac_it[i,t] = ac_it[i,t]-w_ijt_ini[i,j,t]
                        w_it_seq[i,t].append(j)
                        ad_jt[j,t] = 0 
            else:
                break # go to next factory
                
for i in range(0,number_of_facility):
    for j in range(0,number_of_product):
        for t in range(0, time_period):
            try:
                if w_ijt_ini[i,j,t]:
                    pass
            except KeyError:
                w_ijt_ini[i,j,t] = 0
check_feasibility =0
for  j in  range(0,number_of_product):
    for t in range(0, time_period):
        if ad_jt[j,t] >0:
            check_feasibility += 1 
        else:
            pass
if check_feasibility == 0:
    print('Fulfill demand, find the feasible solution! ')
else:
    print('Infeasible solution! ')

Fulfill demand, find the feasible solution! 


In [14]:
# This is the original formulation of the problem

m = gp.Model('Original Problem')
# create decision var.
w_ijt = m.addVars(number_of_facility, number_of_product,time_period,vtype=GRB.CONTINUOUS, name = "w"  )
x_ijt= m.addVars(number_of_facility, number_of_product,time_period,vtype=GRB.BINARY, name = "x"  )
y_ijkt = m.addVars(number_of_facility, number_of_product,number_of_product,time_period,vtype=GRB.BINARY, name = "y" )
z_ijt =  m.addVars(number_of_facility, number_of_product,time_period,vtype=GRB.BINARY, name = "z"  )
v_ijt =  m.addVars(number_of_facility, number_of_product,time_period,vtype=GRB.INTEGER,lb=1, name = "v"  )

# create the obj. function:
m.setObjective(gp.quicksum(r_ij[i,j]* w_ijt[i,j,t] for i in range(0, number_of_facility ) for j in range(0, number_of_product) for t in range(0, time_period)),GRB.MINIMIZE)
# create the constraint:
# constraint 1:
for j in range(0, number_of_product):
    for t in range(0, time_period):
        m.addConstr(gp.quicksum(a_ij[i,j] * w_ijt[i,j,t] for i in range(0, number_of_facility )) == d_jt[j,t],"demand%s,%s"%(j,t))
        
# constrain 2:
for i in range(0, number_of_facility):
    for t in range(0, time_period):
        m.addConstr(gp.quicksum(w_ijt[i,j,t] for j in  range(0, number_of_product)) + 
                    gp.quicksum(q_jk[j,k]*y_ijkt[i,j,k,t] for k in range(0, number_of_product)  for j in  range(0, number_of_product) if j != k ) <= c_it[i,t],"demand_fulfill %s,%s"%(i,t) )
# constaints 3:
for i in range(0, number_of_facility):
    for j  in range(0, number_of_product):
        for t in range(0, time_period):
            m.addConstr(x_ijt[i,j,t] <= z_ijt[i,j,t]+ gp.quicksum(y_ijkt[i,j,k,t]  for k in  range(0, number_of_product) if k != j),"constrint3 %s,%s,%s"%(i,j,t))
# constraint 4:
for i in range(0, number_of_facility):
    for j  in range(0, number_of_product):
        for t in range(0, time_period):
            m.addConstr(a_ij[i,j]*w_ijt[i,j,t] <= d_jt[j,t] * x_ijt[i,j,t],"constraint4 %s,%s,%s"%(i,j,t))
# constraint 5:
for i in range(0, number_of_facility):
    for t in range(0, time_period):
        m.addConstr(gp.quicksum(z_ijt[i,j,t] for j in range(0, number_of_product) ) == 1 ,"constraint5 %s,%s"%(i,t))
#contstraint 6:
for i in range(0, number_of_facility):
    for j in range(0, number_of_product):
        for t in range(0, time_period-1):
            m.addConstr(z_ijt[i,j,t] + gp.quicksum(y_ijkt[i,k,j,t] for k in range(0, number_of_product) if k!= j ) == z_ijt[i,j,t+1]+ gp.quicksum(y_ijkt[i,j,k,t] for k in range(0, number_of_product) if k!= j ),"constraint6 %s,%s,%s"%(i,j,t))
            
# constraint 7:
for i in range(0, number_of_facility):
    for j in range(0, number_of_product):
        for k in range(0, number_of_product):
            for t in range(0, time_period):            
                m.addConstr(v_ijt[i,j,t] +1 - number_of_product+ number_of_product * y_ijkt[i,j,k,t] <= v_ijt[i,k,t],"constraint7 %s,%s,%s,%s"%(i,j,k,t))

m.update()
print('Successful Build model!')
m.Params.timeLimit = 1800.0
m.optimize()
m.objval

Successful Build model!
Set parameter TimeLimit to value 1800
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2376625 rows, 2425500 columns and 15518250 nonzeros
Model fingerprint: 0xba5c6fd7
Variable types: 49500 continuous, 2376000 integer (2326500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 49500 rows and 49500 columns (presolve time = 5s) ...
Presolve removed 49500 rows and 49500 columns (presolve time = 10s) ...
Presolve removed 49500 rows and 49500 columns (presolve time = 15s) ...
Presolve removed 49500 rows and 49500 columns (presolve time = 20s) ...
Presolve removed 49500 rows and 49500 columns
Presolve time: 24.23s
Presolved: 2327125 rows, 2376000 columns, 15468750 nonzeros
Variable types: 49500 continuous, 2326500 integer (2277000 bi

inf

In [3]:
class MasterProblem:
    def __init__(self,column,c_in,a_w_ijtn,a_ij,r_ij,number_of_product,time_period):
        self.model = gp.Model("MasterProblem")
        self.number_of_product = number_of_product
        self.a_w_ijtn = a_w_ijtn 
        self.column = column
        self.subProblemVauleSum = {} 
        self.a_ij =a_ij
        self.r_ij =r_ij
        self.time_period = time_period
        self.number_of_facility =  number_of_facility
        self.c_in = c_in 
    def buildModel(self):
        self.transfer_Column_into_list()
        self.generateVariables()
        self.generateConstraints()
        self.generateObjective()
        self.model.update()
    def generateObjective(self):
        self.model.setObjective(gp.quicksum(self.columnVar[i,n]* self.c_in[i,n] for i in range(0, number_of_facility) for n in range(0,len(self.column[i]))),GRB.MINIMIZE)
        
    def transfer_Column_into_list(self):
        return ([(col,self.column[col][inner])  for col in self.column for inner in self.column[col]])
    def generateVariables(self): 
        self.columnVar = self.model.addVars(self.transfer_Column_into_list(),lb = 0,vtype = GRB.BINARY, name = "lamda")
    def generateConstraints(self):
        for j in range(0, self.number_of_product):
            for t in range(0, time_period):
                self.model.addConstr(gp.quicksum(self.a_w_ijtn[i,j,t,n]*self.columnVar[i,n] for i in range(0, number_of_facility ) for n in range(0,len(self.column[i]))) >= d_jt[j,t],"master_constraint1 %s,%s"%(j,t))
        for i in range(0, self.number_of_facility ):
            self.model.addConstr(gp.quicksum(self.columnVar[i,n] *(-1) for n in range(0,len(self.column[i]) ) )  >= -1,"master_constraint2 %s"%(i)  )
    def solveRelaxedModel(self):
        self.model.update()
        self.relaxModel = self.model.relax() 
        self.relaxModel.optimize() 
    def solveModel(self):
        self.model.optimize()
    def getDuals(self):
        return self.relaxModel.getAttr("Pi", self.relaxModel.getConstrs())
    def addColumn(self, wijt,facility):
        self.column[facility].append(len(self.column[facility]))
        sum_j_t =0
        column_aw= []
        for j in range(0, self.number_of_product):
            for t in range(0, time_period):
                column_aw.append(wijt[facility,j,t] * self.a_ij[facility,j])
                self.a_w_ijtn[facility,j,t,len(self.column[facility]) -1] = wijt[facility,j,t] * self.a_ij[facility,j]
                sum_j_t = sum_j_t +   wijt[facility,j,t] * self.r_ij[facility,j]
        self.c_in[facility,len(self.column[facility]) -1] = sum_j_t
        c = gp.Column()
        c.addTerms(column_aw+[-1 if i==facility else 0 for i in range(0,self.number_of_facility) ],self.model.getConstrs())
        self.model.addVar(vtype = GRB.BINARY, obj = self.c_in[facility,len(self.column[facility]) -1],column = c,name ="lamba[%s,%s]"%(facility,len(self.column[facility]) -1)  )
        self.model.update()                            

class SubProblem:
    def __init__(self, duals,number_of_facility,facility,number_of_product,time_period,r_ij,d_jt,c_it,a_ij,s_jk,
                 q_jk):
        self.duals = duals
        self.facility = facility
        self.number_of_facility = number_of_facility
        self.number_of_product = number_of_product
        self.time_period =time_period
        self.r_ij = r_ij
        self.d_jt =d_jt
        self.a_ij = a_ij
        self.s_jk=s_jk
        self.q_jk = q_jk
        self.c_it = c_it
        self.model = gp.Model("SubProblem")        
    def buildModel(self):
        self.generateVariables()
        self.generateConstraints()
        self.generateObjective()
        self.model.update()
    def generateVariables(self):
        self.SubProblem_wijt = self.model.addVars([self.facility],self.number_of_product,self.time_period,lb=0,vtype = GRB.CONTINUOUS,name = "pricing_wijt")
        self.SubProblem_xijt = self.model.addVars([self.facility],self.number_of_product,self.time_period,vtype = GRB.BINARY, name = "pricing_xijt")
        self.SubProblem_yijkt = self.model.addVars([self.facility],self.number_of_product,self.number_of_product,self.time_period,vtype = GRB.BINARY,name = "pricing_yijkt")
        self.SubProblem_zijt = self.model.addVars([self.facility],self.number_of_product,self.time_period,lb=0,vtype = GRB.BINARY,name = "pricing_zijt")
        self.SubProblem_vijt = self.model.addVars([self.facility],self.number_of_product,self.time_period,lb=1,vtype = GRB.INTEGER,name = "pricing_vijt")
    def generateConstraints(self):
        # constraints 20:
        for t in  range(0, self.time_period):
            self.model.addConstr(gp.quicksum(self.SubProblem_wijt[self.facility,j,t] for j in range(0, self.number_of_product) )  + gp.quicksum(self.q_jk[j,k]*self.SubProblem_yijkt[self.facility,j,k,t] for j in range(0, self.number_of_product) for k in range(0, self.number_of_product))<= self.c_it[self.facility,t],"princing_constraint1 %s,%s"%(self.facility,t))       
        # constaints 21:
        for j  in range(0, self.number_of_product):
            for t in range(0, self.time_period):
                self.model.addConstr(self.SubProblem_xijt[self.facility,j,t] <= self.SubProblem_zijt[self.facility,j,t]+ gp.quicksum(self.SubProblem_yijkt[self.facility,j,k,t]  for k in  range(0, self.number_of_product) if k != j),"pricing_constrint 21 %s,%s,%s"%(self.facility,j,t))
        # constraint 22: # 
        for j  in range(0, self.number_of_product):
            for t in range(0, self.time_period):
                self.model.addConstr(self.a_ij[self.facility,j]*self.SubProblem_wijt[self.facility,j,t] <= self.d_jt[j,t] * self.SubProblem_xijt[self.facility,j,t],"constraint22 %s,%s,%s"%(self.facility,j,t))
        # constraint 23:
        for t in range(0, self.time_period):
            self.model.addConstr(gp.quicksum(self.SubProblem_zijt[self.facility,j,t] for j in range(0, self.number_of_product) ) == 1 ,"constraint23 %s,%s"%(self.facility,t))
        #contstraint 24:
        for j in range(0, self.number_of_product):
            for t in range(0, self.time_period-1):
                self.model.addConstr(self.SubProblem_zijt[self.facility,j,t] + gp.quicksum(self.SubProblem_yijkt[self.facility,k,j,t] for k in range(0, self.number_of_product) if k!= j ) == self.SubProblem_zijt[self.facility,j,t+1]+ gp.quicksum(self.SubProblem_yijkt[self.facility,j,k,t] for k in range(0, self.number_of_product) if k!= j ),"pricing constraint24 %s,%s,%s"%(self.facility,j,t))
        # constraint 25:
        for j in range(0, self.number_of_product):
            for k in range(0, self.number_of_product):
                for t in range(0, self.time_period):                
                    self.model.addConstr(self.SubProblem_vijt[self.facility,j,t] +1 - self.number_of_product+ self.number_of_product * self.SubProblem_yijkt[self.facility,j,k,t] <= self.SubProblem_vijt[self.facility,k,t],"pricing constraint25 %s,%s,%s,%s"%(self.facility,j,k,t))
    def generateObjective(self):
        # partition the dual from the two part
        self.duals_jt= {}
        self.duals_i = {}
        _cnt = 0 
        for j in range(0, self.number_of_product):
            for t in range(0, self.time_period):
                self.duals_jt[j,t] = self.duals[_cnt]
                _cnt +=1
        for i in range(0,self.number_of_facility):
            self.duals_i[i] = self.duals[_cnt]
            _cnt +=1            
        self.model.setObjective(gp.quicksum(self.r_ij[self.facility,j] *self.SubProblem_wijt[self.facility,j,t] 
                                            for j in range(0, self.number_of_product)  for t in range(0, self.time_period) ) -
                                gp.quicksum(self.duals_jt[j,t]*self.a_ij[self.facility,j]*
                                            self.SubProblem_wijt[self.facility,j,t] for j in range(0, self.number_of_product) 
                                            for t in range(0, self.time_period) ) + self.duals_i[self.facility],GRB.MINIMIZE)
    def solveModel(self,timeLimit ,EPSILON):
        self.model.setParam('TimeLimit', timeLimit)
        self.model.setParam('MIPGAP', EPSILON)
        self.model.optimize()
    def obtain_column(self):
        wijt_solution = {}
        cnt = 0 
        for j in range(0,self.number_of_product):
            for t in range(0,self.time_period):
                wijt_solution[self.facility,j,t] = self.model.getVars()[cnt].X
                cnt = cnt +1 
        return wijt_solution
    def obtain_pricing_solution(self):
        return self.model.objval, self.model.getAttr("X",self.model.getVars()),self.model.getAttr("VarName",self.model.getVars())    
        
class CreateInitialSolution:
    def __init__(self, ini_wijt,number_of_facility,number_of_product,time_period,a_ij,r_ij):
        self.a_ij =a_ij
        self.r_ij =r_ij
        self.ini_wijt = ini_wijt
        self.number_of_facility = number_of_facility
        self.number_of_product = number_of_product
        self.time_period = time_period
    def crateInitialColumn(self):
        column = {}
        c_in = {}
        a_w_ijtn= {}
        for i in range(0,self.number_of_facility):
            column[i] = []
            column[i].append(0)
            c_in[i,0] = 0
            sum_j_t = 0
            for j in range(0,self.number_of_product):
                for t in range(0,self.time_period):
                    a_w_ijtn[i,j,t,0] = self.ini_wijt[i,j,t]* self.a_ij[i,j]
                    sum_j_t = sum_j_t + (self.ini_wijt[i,j,t]* self.r_ij[i,j])
            c_in[i,0] = sum_j_t # store the coefficient
        return column,c_in,a_w_ijtn



In [4]:
initial = CreateInitialSolution(w_ijt_ini,number_of_facility,number_of_product,time_period,a_ij,r_ij)
initial_column,c_in,a_w_ijtn =initial.crateInitialColumn()
masterProblem = MasterProblem(initial_column,c_in,a_w_ijtn,a_ij,r_ij,number_of_product,time_period)
masterProblem.buildModel()
masterProblem.solveRelaxedModel()
count = 0 

direction = True
cnt_column = 0
obj_val = []
# Doing the column generation procedure, consider the finite column (column <=200) and also avoid generate the same colunm
while direction == True and cnt_column <= 200:
    direcition = False
    if count == 0 :
        masterdual = masterProblem.getDuals()
        pass
    else:            
        masterProblem.solveRelaxedModel()
        print('OBJ:', masterProblem.relaxModel.objval)
        obj_val.append(masterProblem.relaxModel.objval)
        masterdual = masterProblem.getDuals()
    count =1 

    for i in range(0,initial.number_of_facility):
        subProblem = SubProblem(masterdual,number_of_facility,i,number_of_product,time_period,r_ij,d_jt,c_it,a_ij,s_jk,
                     q_jk)
        subProblem.buildModel()
        subProblem.solveModel(130,0.10)
        if subProblem.obtain_pricing_solution()[0] < 0:
            cnt_column += 1 
            direction = True
            subProblem.obtain_pricing_solution()
            price_solution =subProblem.obtain_column()
            try:
                masterProblem.subProblemVauleSum[i].append(sum([i.x for i in subProblem.model.getVars() if i.VarName[0:12] == 'pricing_wijt' ]))
            except KeyError:
                masterProblem.subProblemVauleSum[i] = []
                masterProblem.subProblemVauleSum[i].append(sum([i.x for i in subProblem.model.getVars()  if i.VarName[0:12] == 'pricing_wijt']))
                
            masterProblem.addColumn(price_solution,i)
        else:
            pass
masterProblem.model.Params.timeLimit = 600
masterProblem.solveRelaxedModel()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 955 rows, 55 columns and 1403 nonzeros
Model fingerprint: 0x70abe8cf
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 955 rows and 55 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9977090e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.997709000e+06
Set parameter TimeLimit to value 130
Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Op

Presolve removed 4631 rows and 4675 columns
Presolve time: 0.32s
Presolved: 38564 rows, 39425 columns, 229555 nonzeros
Variable types: 23 continuous, 39402 integer (38545 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Extra simplex iterations after uncrush: 10
Concurrent spin time: 0.00s

Solved with primal simplex

Root relaxation: objective -1.353953e+06, 1242 iterations, 0.31 seconds (0.31 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -1353952.7    0   49    0.00000 -1353952.7      -     -    1s
H    0     0                    -649134.5000 -1353952.7   109%     -    1s
H    0     0                    -758119.5577 -1353952.7  78.6%     -    2s
H    0     0                    -903055.0000 -1353952.7  49.9%     -    2s
     0     0 -1326957.4    0  125 -903055.00 -1326957.4  46.9%     -    2s
H    0     0 

Set parameter TimeLimit to value 130
Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 43195 rows, 44100 columns and 282150 nonzeros
Model fingerprint: 0x73d328de
Variable types: 900 continuous, 43200 integer (42300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [5e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 0.0000000
Presolve removed 4631 rows and 4675 columns
Presolve time: 0.31s
Presolved: 38564 rows, 39425 columns, 229555 nonzeros
Variable types: 23 continuous, 39402 integer (38545 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Extra simplex iterations after uncrush: 10
Concurrent spin time: 0.00s

Solved with primal simplex

Root relaxation: objective -1.353854e+06, 1188 iterations, 0.35


Cutting planes:
  Gomory: 7
  MIR: 32
  Flow cover: 2
  Zero half: 4
  RLT: 1

Explored 1 nodes (4046 simplex iterations) in 4.06 seconds (1.93 work units)
Thread count was 8 (of 8 available processors)

Solution count 6: -1.24732e+06 -1.15439e+06 -1.12102e+06 ... 0

Optimal solution found (tolerance 1.00e-01)
Best objective -1.247318396862e+06, best bound -1.321321339111e+06, gap 5.9330%
Set parameter TimeLimit to value 130
Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 43195 rows, 44100 columns and 282150 nonzeros
Model fingerprint: 0xdff0472f
Variable types: 900 continuous, 43200 integer (42300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [5e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 0.0000000
Presolve removed 4631 rows and 4675 colu

     0     0 -1316260.9    0  156 -1198025.6 -1316260.9  9.87%     -    4s

Cutting planes:
  Learned: 1
  Gomory: 11
  Lift-and-project: 3
  MIR: 72
  Flow cover: 5
  Zero half: 5
  Mod-K: 3
  RLT: 2

Explored 1 nodes (5043 simplex iterations) in 4.76 seconds (2.39 work units)
Thread count was 8 (of 8 available processors)

Solution count 5: -1.19803e+06 -1.1948e+06 -917492 ... 0

Optimal solution found (tolerance 1.00e-01)
Best objective -1.198025572205e+06, best bound -1.316260865726e+06, gap 9.8692%
Set parameter TimeLimit to value 130
Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 43195 rows, 44100 columns and 282150 nonzeros
Model fingerprint: 0x78288829
Variable types: 900 continuous, 43200 integer (42300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RH

...

Thread count was 8 (of 8 available processors)

Solution count 5: -1.98219e+06 -1.64909e+06 -1.0243e+06 ... 0

Optimal solution found (tolerance 1.00e-01)
Best objective -1.982193551284e+06, best bound -2.135619900132e+06, gap 7.7402%
Set parameter TimeLimit to value 130
Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 43195 rows, 44100 columns and 282150 nonzeros
Model fingerprint: 0xb176c899
Variable types: 900 continuous, 43200 integer (42300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+01, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 0.0000000
Presolve removed 4633 rows and 4677 columns
Presolve time: 0.39s
Presolved: 38562 rows, 39423 columns, 229506 nonzeros
Variable types: 19 continuous, 39404 integer (38544 binary)

Deterministic conc

  Objective range  [1e+01, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 0.0000000
Presolve removed 4633 rows and 4677 columns
Presolve time: 0.53s
Presolved: 38562 rows, 39423 columns, 229506 nonzeros
Variable types: 19 continuous, 39404 integer (38544 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Extra simplex iterations after uncrush: 4
Concurrent spin time: 0.00s

Solved with primal simplex

Root relaxation: objective -2.192201e+06, 1766 iterations, 0.39 seconds (0.36 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -2192201.1    0   71    0.00000 -2192201.1      -     -    1s
H    0     0                    -1007981.000 -2192201.1   117%     -    1s
H    0     0                    -1175176.365 -2192201.1  86.5%     -    5s
H    0     0    

Set parameter TimeLimit to value 130
Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 43195 rows, 44100 columns and 282150 nonzeros
Model fingerprint: 0x707be05f
Variable types: 900 continuous, 43200 integer (42300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+01, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 0.0000000
Presolve removed 4633 rows and 4677 columns
Presolve time: 0.32s
Presolved: 38562 rows, 39423 columns, 229506 nonzeros
Variable types: 19 continuous, 39404 integer (38544 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Extra simplex iterations after uncrush: 4
Concurrent spin time: 0.00s

Solved with primal simplex

Root relaxation: objective -2.197827e+06, 1871 iterations, 0.39 

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 0.0000000
Presolve removed 4633 rows and 4677 columns
Presolve time: 0.33s
Presolved: 38562 rows, 39423 columns, 229506 nonzeros
Variable types: 19 continuous, 39404 integer (38544 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Extra simplex iterations after uncrush: 5
Concurrent spin time: 0.01s

Solved with primal simplex

Root relaxation: objective -2.185899e+06, 1818 iterations, 0.64 seconds (0.40 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -2185899.3    0   72    0.00000 -2185899.3      -     -    1s
H    0     0                    -1018422.000 -2185899.3   115%     -    1s
H    0     0                    -1171201.329 -2185899.3  86.6%     -    2s
     0     0 -2145971.6    0   89 -1171201.3 -2145


Extra simplex iterations after uncrush: 5
Concurrent spin time: 0.00s

Solved with primal simplex

Root relaxation: objective -2.188783e+06, 1690 iterations, 0.46 seconds (0.35 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -2188783.0    0   59    0.00000 -2188783.0      -     -    1s
H    0     0                    -1399563.000 -2188783.0  56.4%     -    1s
H    0     0                    -1839125.434 -2188783.0  19.0%     -    1s
H    0     0                    -1895203.682 -2188783.0  15.5%     -    2s
H    0     0                    -1901826.458 -2188783.0  15.1%     -    2s
H    0     0                    -1961028.891 -2188783.0  11.6%     -    2s
     0     0 -2147789.5    0  201 -1961028.9 -2147789.5  9.52%     -    4s

Cutting planes:
  Gomory: 9
  MIR: 42
  Flow cover: 2
  Zero half: 8
  RLT: 2
  Relax-and-lift: 4

Explored 1 nodes (5091 simplex it

In [9]:
# Solve the master problem (i.e. restricted Master Problem)
masterProblem.model.Params.timeLimit = 800
masterProblem.solveModel()
masterProblem.model.objval

Set parameter TimeLimit to value 800
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 955 rows, 275 columns and 4841 nonzeros
Model fingerprint: 0x87560671
Variable types: 0 continuous, 275 integer (275 binary)
Coefficient statistics:
  Matrix range     [9e-05, 7e+02]
  Objective range  [1e+03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective 2997709.0000
Presolve removed 955 rows and 275 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 2.99771e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.997709000000e+06, best bound 2.997709000000e+06, gap 0.0000%


2997709.0

In [24]:
print('The best bound found by solver is:', m.ObjBound)
print('The feasible solution found by column generation procedure is:', masterProblem.model.objval)
print('The GAP is:',masterProblem.model.objval/(masterProblem.model.objval-m.ObjBound))

The best bound found by solver is: 2777490.0000000014
The feasible solution found by column generation procedure is: 2997709.0
The GAP is: 13.612399475068086
